# Testing out creating KN injections

I first have to convert my GW injections to a `.dat` file that is compatible with NMMA. I am recreating something like [this example file](https://github.com/nuclear-multimessenger-astronomy/nmma/blob/main/example_files/sim_events/injections.dat).

In [4]:
import os
import json
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline  
import nmma.em.analysis
import corner

params = {"axes.grid": True,
        "text.usetex" : True,
        "font.family" : "serif",
        "ytick.color" : "black",
        "xtick.color" : "black",
        "axes.labelcolor" : "black",
        "axes.edgecolor" : "black",
        "font.serif" : ["Computer Modern Serif"],
        "xtick.labelsize": 16,
        "ytick.labelsize": 16,
        "axes.labelsize": 16,
        "legend.fontsize": 16,
        "legend.title_fontsize": 16,
        "figure.titlesize": 16}

plt.rcParams.update(params)

# Improved corner kwargs
default_corner_kwargs = dict(bins=40, 
                        smooth=1., 
                        show_titles=False,
                        label_kwargs=dict(fontsize=16),
                        title_kwargs=dict(fontsize=16), 
                        color="blue",
                        # quantiles=[],
                        # levels=[0.9],
                        plot_density=True, 
                        plot_datapoints=False, 
                        fill_contours=True,
                        max_n_ticks=4, 
                        min_n_ticks=3,
                        save=False)

from bilby.gw.conversion import chirp_mass_and_mass_ratio_to_component_masses

In [5]:
def get_dat_file(outdir):
    """
    Convert my set of GW injections to a .dat file to be consumed by NMMA.
    """
    # Runs contain the directories' names, e.g. `injection_123`
    
    runs = os.listdir(outdir)
    my_dict = {"simulation_id": [],
               "longitude": [],
               "latitude": [],
               "inclination": [],
               "distance": [],
               "mass_1": [],
               "mass_2": [],
               "spin1z": [],
               "spin2z": []}
    
    # Go over the runs and save each of these:
    for run in runs:
        # Add the simulation ID
        simulation_id = run.split("_")[-1]
        my_dict["simulation_id"].append(int(simulation_id))
        
        # Load the json to get the other values
        this_dir = os.path.join(outdir, run)
        file = os.path.join(this_dir, "config.json")
        with open(file, "r") as f:
            config = json.load(f)
            
            # Masses
            mc, q = config["M_c"], config["q"]
            m1, m2 = chirp_mass_and_mass_ratio_to_component_masses(mc, q)
            my_dict["mass_1"].append(m1)
            my_dict["mass_2"].append(m2)
            
            # Spins
            my_dict["spin1z"].append(config["s1_z"])
            my_dict["spin2z"].append(config["s2_z"])
            
            # External:
            my_dict["inclination"].append(np.arccos(config["cos_iota"]))
            my_dict["distance"].append(config["d_L"])
            my_dict["longitude"].append(config["ra"])
            my_dict["latitude"].append(np.arcsin(config["sin_dec"]))
            
    return my_dict

In [10]:
def edit_injection_json(filename_json):
    """
    For some reason there is an issue with KNtheta, remove that from the json and save it again. Perhaps will have to do something else in the future, so a separate function.
    """
    
    # Load original
    with open(filename_json, "r") as f:
        config = json.load(f)
    
    # Make the desired edits
    try: 
        config["injections"]["content"].pop("KNtheta")
    except KeyError:
        print("KeyError was given, perhaps KNtheta was already removed?")
        
    # Save it again
    with open(filename_json, "w") as f:
        json.dump(config, f)

In [7]:
my_dict = get_dat_file("../injections_tidal/outdir/")
df = pd.DataFrame(my_dict)
# df # to show
df.to_csv("injections.dat", sep=" ", index=False)

## Check json

**NOW: Run the create-injection**. Then here we will check that output in more detail. Noticed that there is an issue with the KNtheta, so I will remove that manually here. 

In [11]:
filename = "./outdir/injection_Bu2022Ye.json"
edit_injection_json(filename)

KeyError was given, perhaps KNtheta was already removed?


In [9]:
with open(filename, "r") as f:
    config = json.load(f)
    injections = config["injections"]["content"]
    # print(injections.keys())

print(len(injections["mass_1"]))

for key, val in injections.items():
    print(key)
    # print(val)

21
simulation_id
longitude
latitude
inclination
distance
mass_1
mass_2
spin1z
spin2z
timeshift
luminosity_distance
inclination_EM
log10_mej_dyn
vej_dyn
Yedyn
log10_mej_wind
vej_wind
alpha
ratio_zeta
ratio_epsilon
cos_theta_jn
geocent_time
phi_jl
phi_12
theta_jn
redshift
mass_1_source
mass_2_source
TOV_mass
TOV_radius
lambda_1
lambda_2
radius_1
radius_2
R_16
R_14
log10_mej
log10_E0


# === Archive ===

Something seemed off with the NMMA runs, but turns out, it is most likely just issues with the realistic KN injections -- you might have to keep on resampling to get a decent, detectable LC ...

In [7]:
# # The parameters for the injection
# my_params = ['luminosity_distance', 'inclination_EM', 'log10_mej_dyn', 'vej_dyn', 'Yedyn', 'log10_mej_wind', 'vej_wind', 'alpha', 'ratio_zeta', 'ratio_epsilon', 'cos_theta_jn']

# # The file with the test run
# file = "/home/urash/twouters/projects/jim_H0/KN/outdir/BNS/0/injection_Bu2022Ye_0_result.json"

# with open(file, "r") as f:
#     result = json.load(f)
#     posterior = result["posterior"]["content"]
#     my_posterior_values = np.array([posterior[key] for key in my_params])
# keys = posterior.keys()

# # Make a corner plot
# corner.corner(my_posterior_values.T, labels=my_params, **default_corner_kwargs)
# plt.show()

## (Extra:) Analyze some variables for the KN runs setup

Used to check what kind of priors I should use

In [8]:
# # Plot histograms for each variable
# for key in my_dict.keys():
#     print(key)
#     print(my_dict[key])
#     if key == "simulation_id":
#         continue
#     plt.figure()
#     plt.hist(my_dict[key], bins=20)
#     plt.title(key)
#     plt.show()